In [14]:
import torch
import math

Рассмотрим стандартную нормальную иницализацию

In [16]:
x = torch.randn(128)
for i in range(100):
    a = torch.randn(128,128)
    x = a @ x

x.mean(), x.std()

(tensor(nan), tensor(nan))

Когда же у нас появляются такие веса?

In [17]:
x = torch.randn(128)
for i in range(100):
    a = torch.randn(128,128)
    x = a @ x
    if torch.isnan(x.mean()):
        break

i

35

Теперь уменьшим нашу инициализацию

In [18]:
x = torch.randn(128)
for i in range(100):
    a = torch.randn(128,128) * 0.01
    x = a @ x

x.mean(), x.std()

(tensor(0.), tensor(0.))

Стандартное отклонение дисперсии выхода каждого слоя приблизительно равно корню из количества входных нейронов.

In [20]:
mean, var = 0, 0 
for i in range(1000):
    x = torch.randn(128)
    a = torch.randn(128,128)
    y = a @ x
    mean += y.mean().item()
    var += y.pow(2).mean().item()
    
math.sqrt(var/1000), math.sqrt(128)

(11.33242086581917, 11.313708498984761)

Значения до применения активаций  $z_j$ для нейрона $ j $:


$z_j = \sum_{i=1}^{n_{\text{in}}} W_{ji} x_i$

$x_i$ и $W_{ji}$  **независимы**, поэтому:

$\text{Var}(z_j) = \sum_{i=1}^{n_{\text{in}}} \text{Var}(W_{ji} x_i)$


Перепишем:

$\text{Var}(z_j) = \sum_{i=1}^{n_{\text{in}}} \sigma_W^2 \sigma_x^2 = n_{\text{in}} \sigma_W^2 \sigma_x^2$ (сумма дисперсии по каждой строке для $W$)

Мы хотим, чтобы распреление данных входа было равно распределению на выходе:

$\text{Var}(z_j) = \sigma_x^2$

Выписывая $n_{\text{in}} \sigma_W^2 \sigma_x^2 = \sigma_x^2 $,решаем для $ \sigma_W^2 $:

$\sigma_W^2 = \frac{1}{n_{\text{in}}} $



Проверим на 1

In [21]:
mean, var = 0, 0 
for i in range(1000):
    x = torch.randn(1)
    a = torch.randn(1)
    y = a * x
    mean += y.mean().item()
    var += y.pow(2).item()
    
math.sqrt(var/1000)

1.015961416795791

Вывод - давайте нормализовать каждый слой на корень из количества входных нейронов

In [22]:
x = torch.randn(128)
for i in range(1000):
    a = torch.randn(128,128) * math.sqrt(1/128)
    x = a @ x
    
x.mean(), x.std()

(tensor(-0.0072), tensor(0.1790))

In [23]:
x

tensor([ 2.5014e-01,  1.3446e-01, -2.3238e-01,  8.6074e-03, -6.0112e-02,
         4.2212e-01, -2.5839e-01, -9.9810e-03,  2.5922e-01,  2.2621e-01,
        -2.6025e-01, -2.5173e-01,  7.3547e-02, -1.7332e-02, -1.2269e-01,
        -4.2060e-01,  2.1054e-01,  7.4447e-02,  2.8893e-01,  2.2744e-01,
         9.4124e-02, -2.4312e-02,  2.9872e-01,  2.4650e-01,  5.8994e-02,
         5.7025e-02, -1.9131e-01, -6.1351e-02,  8.8850e-02, -2.4190e-01,
         5.2480e-02,  3.8885e-01,  6.6200e-03, -2.3647e-01,  2.1772e-01,
         1.6629e-01,  2.5183e-02,  6.6195e-02, -1.4505e-01, -1.6593e-01,
         2.4484e-01,  2.0980e-01,  1.3511e-01, -3.7339e-02,  7.5982e-02,
         1.0937e-01,  8.1519e-02, -3.1813e-01,  5.9521e-02,  8.1355e-02,
         1.0061e-01, -1.6339e-01, -1.8832e-01, -1.8787e-01, -3.1846e-01,
         1.9250e-01, -7.3521e-02,  1.4589e-01,  3.3394e-02, -1.0247e-01,
         2.9026e-02, -1.5536e-01,  2.7931e-02, -4.3419e-02, -1.8210e-01,
        -2.6615e-01, -1.1160e-01, -3.2862e-04, -3.1

А раньше использовали равномерное распределение

In [26]:
x = torch.randn(128)

for i in range(100):
    a = torch.Tensor(128, 128).uniform_(-1,1) * math.sqrt(1/128)
    x = torch.tanh(a@x)
    
x.mean(), x.std()

(tensor(3.2702e-26), tensor(4.6586e-25))

Попробуем наш подход

In [27]:
x = torch.randn(128)
for i in range(1000):
    a = torch.randn(128,128) * math.sqrt(1/128)
    x = torch.tanh(a @ x)
    
x.mean(), x.std()

(tensor(-8.5019e-05), tensor(0.0014))

Статья http://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf?hc_location=ufi - позволила лучше работать с нелинейностями

In [28]:
def xavier_uniform(n,k):
    return torch.Tensor(n,k).uniform_(-1,1)*math.sqrt(6/(n+k))

def xavier_normal(n,k):
    return torch.normal(0, math.sqrt(3/(n+k)), size=(n, k))

x = torch.randn(128)

for i in range(100):
    a = xavier_uniform(128, 128)
    x = torch.tanh(a@x)
    
x.mean(), x.std()

(tensor(0.0015), tensor(0.0283))

Пока не изобрели ReLu - все опять поплыло

In [29]:
x = torch.randn(128)

for i in range(100):
    a = xavier_uniform(128, 128)
    x = (a@x).clamp_min(0)
    
x.mean(), x.std()

(tensor(2.4331e-16), tensor(3.4858e-16))

В статье Kaiming He https://arxiv.org/pdf/1502.01852v1.pdf предлагает домножать стандартное отклонение на $\sqrt 2$

In [46]:
def kaiming(n,k):
    return torch.normal(0, math.sqrt(2/n), size=(n, k))

x = torch.randn(128)

for i in range(100):
    a = kaiming(128, 128)
    x = (a@x).clamp_min(0)
    
x.mean(), x.std()

(tensor(0.0624), tensor(0.0949))

Также неплохо работает с другими нелинейностями

In [47]:
x = torch.randn(128)

for i in range(100):
    a = kaiming(128, 128)
    x = torch.tanh(a@x)
    
x.mean(), x.std()

(tensor(0.0034), tensor(0.5260))